In [1]:
import pickle
import time
import tqdm
from dijkstra import *
from ch import *
from bidij import *
from astar import *

In [2]:
n = 49109 # nodes
m = 60512 # edges
q = 1000  # queries
with open('edges.pickle','rb') as f:
    edges   = pickle.load(f)

with open('coords.pickle','rb') as f:
    coords  = pickle.load(f)

with open('queries.pickle','rb') as f:
    queries = pickle.load(f)

assert len(edges) == m
assert len(coords) == n
assert len(queries) == q

In [3]:
adj_bi  = [[[] for _ in range(n)], [[] for _ in range(n)]]
cost_bi = [[[] for _ in range(n)], [[] for _ in range(n)]]
adj = [[] for _ in range(n)]
cost = [[] for _ in range(n)]
for e in edges:
    u,v,c = e[0],e[1],e[2]
    adj_bi[0][u-1].append(v-1)
    cost_bi[0][u-1].append(c)
    adj_bi[1][v-1].append(u-1)
    cost_bi[1][v-1].append(c)
    adj[u-1].append(v-1)
    cost[u-1].append(c)
    
x = [c[0] for c in coords]
y = [c[1] for c in coords]

## Dijkstra

In [4]:
t1 = time.time()
for i in tqdm.tqdm(range(q)):
    u,v = queries[i][0],queries[i][1]
    dijkstra(adj, cost, u-1, v-1)
t2 = time.time()

print(f"Dijkstra: {t2-t1}")

100%|██████████| 1000/1000 [00:33<00:00, 29.87it/s]

Dijkstra: 33.48875617980957


## Bidirectional Dijkstra

In [5]:
bidij = BiDij(n)
bidij.define_values(adj_bi, cost_bi)
t1 = time.time()
for i in tqdm.tqdm(range(q)):
    u,v = queries[i][0],queries[i][1]
    bidij.query(adj_bi, cost_bi, u-1, v-1)
t2 = time.time()
print(f"Bidirectional Dijkstra: {t2-t1}")

100%|██████████| 1000/1000 [00:05<00:00, 170.98it/s]

Bidirectional Dijkstra: 5.849478006362915


## A* Algorithm

In [6]:
astar = AStar(n, adj, cost, x, y)
t1 = time.time()
for i in tqdm.tqdm(range(q)):
    u,v = queries[i][0],queries[i][1]
    astar.query(u-1, v-1)
t2 = time.time()
print(f"A*: {t2-t1}")

100%|██████████| 1000/1000 [04:12<00:00,  3.96it/s]

A*: 252.84441208839417


## Contraction Hierarchies

In [7]:
t1 = time.time()
ch = DistPreprocessSmall(n, adj_bi, cost_bi)
t2 = time.time()
for i in tqdm.tqdm(range(q)):
    u,v = queries[i][0],queries[i][1]
    ch.query(u-1, v-1)
t3 = time.time()
print(f"CH-preprocess: {t2-t1}")
print(f"CH-query: {t3-t2}")

100%|██████████| 1000/1000 [00:00<00:00, 7895.00it/s]

CH-preprocess: 10.156567335128784
CH-query: 0.12765955924987793
